In [1]:
import openai
import pandas as pd
import json
import os
from collections import deque
import random

In [2]:
def generate_random_maze(n):
    maze = [[random.choice(['x', 'o']) for _ in range(n)] for _ in range(n)]
    maze[0][0] = 's'  # Set start position
    maze[n-1][n-1] = 'e'  # Set end position
    return maze

def solve_maze_corrected(maze, start, start_direction):
    n = len(maze)  # Determine the size of the maze dynamically
    end = (n - 1, n - 1)  # Dynamically determine the end position
    queue = deque([(start, start_direction, [])])
    visited = set([(start, start_direction)])
    
    while queue:
        (current, direction, path) = queue.popleft()
        
        if current == end:
            return path
        
        for i, (di, dj) in enumerate(directions):
            next_direction = (direction + i) % 4  # Adjust direction based on turn
            new_pos = (current[0] + directions[next_direction][0], current[1] + directions[next_direction][1])
            
            # Validate position
            if (0 <= new_pos[0] < n and 0 <= new_pos[1] < n and  # Inside bounds
                maze[new_pos[0]][new_pos[1]] in ('o', 'e') and  # Valid cell
                (new_pos, next_direction) not in visited):  # Not visited
                
                # Generate new path
                new_path = path[:]
                
                # Add turns to align direction
                turn_steps = (i if i <= 2 else -1)
                if turn_steps > 0:
                    new_path.extend(["left"] * turn_steps)
                elif turn_steps < 0:
                    new_path.extend(["right"] * abs(turn_steps))
                
                # Move forward
                new_path.append("forward")
                
                # Add to queue
                queue.append((new_pos, next_direction, new_path))
                visited.add((new_pos, next_direction))
    
    return ["no path"]

directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
start = (0, 0)  
start_direction = 0 


# Create 10 CSV files, each containing one maze
output_directory = "test_mazes"
os.makedirs(output_directory, exist_ok=True)

maze_size = 5  # Size of the maze
i = 0
while i < 1000:
    maze = generate_random_maze(maze_size)
    corrected_solution = solve_maze_corrected(maze, start, start_direction)
    if corrected_solution == ["no path"]:
        pass
    else:
        df = pd.DataFrame(maze)
        file_path = os.path.join(output_directory, f"maze_{i}.csv")
        df.to_csv(file_path, index=False, header=False)
        i += 1

In [65]:
json_file = r'train_test5.json'

try:
    with open(json_file, "r") as file:
        dataset = json.load(file) 
except FileNotFoundError:
    dataset = []  

print("initial length: " + str(dataset.__len__()))

initial length: 0


In [ ]:
json_file = r'chatGPT_training_data.json'

try:
    with open(json_file, "r") as file:
        dataset = json.load(file) 
except FileNotFoundError:
    dataset = []  

print("initial length: " + str(dataset.__len__()))


# Set to store unique (maze, solution) combinations
unique_combinations = set((json.dumps(entry["maze"]), json.dumps(entry["solution"]["instructions"])) for entry in dataset)

i = 0
max_entries = 500 #length of dataset
no_path_limit = int(max_entries * 0.1)  # Target 20% "no path" mazes
no_path_count = sum(1 for entry in dataset if entry["solution"]["instructions"] == ["no path"])

while i < max_entries:
    n = 5
    new_maze = generate_random_maze(n)
    corrected_solution = solve_maze_corrected(new_maze, start, start_direction)

    new_entry = {
        "maze": new_maze,
        "solution": {"instructions": corrected_solution}
    }   

    # Serialize the maze and solution to check for duplicates
    serialized_maze = json.dumps(new_maze)
    serialized_solution = json.dumps(corrected_solution)
    combination = (serialized_maze, serialized_solution)

    if corrected_solution == ["no path"]:
        if no_path_count < no_path_limit and combination not in unique_combinations:
            dataset.append(new_entry)
            unique_combinations.add(combination)
            no_path_count += 1
            i += 1
    else:
        if combination not in unique_combinations:
            dataset.append(new_entry)
            unique_combinations.add(combination)
            i += 1

with open(json_file, "w") as file:
    json.dump(dataset, file, indent=4)

print("new length: " + str(len(dataset)))

initial length: 0
new length: 500


In [5]:
# transform the dataset into training data format for OpenAI Fine Tuning

training_data = []
for i, item in enumerate(dataset, 1):
    maze_string = "\n".join([" ".join(row) for row in item["maze"]])
    output = json.dumps(item["solution"], indent=4)
    
    prompt = f"""
    Solve the given maze following the rules and building blocks provided.

    Building Blocks:
    - "forward": Moves block one cell in the direction the block is currently facing.
    - "backward": Moves block one cell in the opposite direction the block is currently facing.
    - "left": Rotates block 90 degrees counterclockwise in place (does not move cells).
    - "right": Rotates block 90 degrees clockwise in place (does not move cells).

    Maze File:
    This is a 5 x 5 grid where:
    - 's' represents the start cell.
    - 'e' represents the end cell.
    - 'x' represents an occupied cell.
    - 'o' represents an open path.

    Here is the maze you need to solve:
    {maze_string}

    Rules:
    1. The block starts at the 's' cell facing down (The start cell is (0,0) and is facing in the direction of (1,0)).
    2. You can only move into 'o' cells. You cannot move into 'x' cells.
    3. The block cannot move outside the maze boundaries.
    4. The goal is to navigate from 's' to 'e' only using the building blocks (forward, backward, left, and right).

    Output Requirement:
    - Provide a JSON response with the list of instructions to solve the maze.

    Example Output:
    {{
        "instructions": ["forward", "left", "forward", "right", ...]
    }}

    If no path exists, return:
    {{
        "instructions": ["no path"]
    }}

    Only return the instructions list in the JSON response.
    """

    entry = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": json.loads(output)}
        ]
        
    }

    training_data.append(entry)


# Write the training data to a file in JSONL format
with open("training_data.jsonl", "w") as file:
    for entry in training_data:
        file.write(json.dumps(entry) + "\n")

print("Training data saved to training_data.jsonl")

Training data saved to training_data.jsonl


In [6]:
# Properly format the JSONL file so it is acceptable for OpenAI's API
input_file = "training_data.jsonl"
output_file = "training_data_cleaned.jsonl"

def wrap_content_in_quotations(line):
    # Parse the JSON line
    data = json.loads(line)
    
    # Locate the third message and wrap its "content" in quotations
    for message in data.get("messages", []):
        if message["role"] == "assistant" and isinstance(message["content"], dict):
            # Serialize the dictionary to a JSON string
            message["content"] = json.dumps(message["content"])
    
    # Return the modified JSON as a string
    return json.dumps(data)

# Read and process the JSONL file
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        modified_line = wrap_content_in_quotations(line)
        outfile.write(modified_line + "\n")

print(f"Processed file saved to {output_file}")

Processed file saved to training_data_cleaned.jsonl


##### To actually train the model it is easiest to create a fine tuning instance on your dashboard on OpenAI's web browsing platform. You can drop your jsonl file (training data) here and allow it to train. This will take some time. Once it is done you can copy the output model ID string and use it as a model. 